In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("dataset.csv")
df.head()

,No,Date,C/code,Customer_name,Customer_address,Item_code,Product_name,Orignal_price,CTN,PCS,Total_discount,Discount_price,PCS_in_CTN,Price_per_PCS,Discount_per_piece,Month,Price Range,Has_Discount
0,3,2021-09-10,9295,BIG BITE FAST FOOD,WARSAK ROAD,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500,1,10,1000,13500,10,1450.0,100.0,9,Low,1
1,3,2021-09-10,9295,BIG BITE FAST FOOD,WARSAK ROAD,1771,DEEN'S MOZRELLA CHEESE BLOCK,19250,0,5,375,9250,10,1925.0,75.0,9,Low,1
2,4,2021-09-10,8992,PIZZA FLAME,IBRAHIM MARKET,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500,0,2,200,2700,10,1450.0,100.0,9,Low,1
3,4,2021-09-10,8992,PIZZA FLAME,IBRAHIM MARKET,1771,DEEN'S MOZRELLA CHEESE BLOCK,19250,0,2,150,3700,10,1925.0,75.0,9,Low,1
4,5,2021-09-10,9903,CHIL & GRILL 0312.9553655,MOHMAD ZAI,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500,0,2,200,2700,10,1450.0,100.0,9,Low,1


In [3]:
df = pd.read_csv('dataset.csv',header=1,names=["No","Date","C_id","Customer_name","Customer_address","I_code","Product_name","CTN_price","CTN","PCS","Total_discount","Net_value"],parse_dates=["Date"])
df.head()

/tmp/ipykernel_11017/1950842724.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('dataset.csv',header=1,names=["No","Date","C_id","Customer_name","Customer_address","I_code","Product_name","CTN_price","CTN","PCS","Total_discount","Net_value"],parse_dates=["Date"])


No  \
3 2021-09-10 9295 BIG BITE FAST FOOD        WARSAK ROAD    1771    DEEN'S MOZRELLA CHEESE BLOCK   
4 2021-09-10 8992 PIZZA FLAME               IBRAHIM MARKET 1770  DEEN'S CHADDER CHEESE BLOCK 2K   
                                                           1771    DEEN'S MOZRELLA CHEESE BLOCK   
5 2021-09-10 9903 CHIL & GRILL 0312.9553655 MOHMAD ZAI     1770  DEEN'S CHADDER CHEESE BLOCK 2K   
                                                           1771    DEEN'S MOZRELLA CHEESE BLOCK   

                                                                  Date  C_id  \
3 2021-09-10 9295 BIG BITE FAST FOOD        WARSAK ROAD    1771  19250     0   
4 2021-09-10 8992 PIZZA FLAME               IBRAHIM MARKET 1770  14500     0   
                                                           1771  19250     0   
5 2021-09-10 9903 CHIL & GRILL 0312.9553655 MOHMAD ZAI     1770  14500     0   
                                                           1771  19250     0   

                                                                 Customer_name  \
3 2021-09-10 9295 BIG BITE FAST FOOD        WARSAK ROAD    1771              5   
4 2021-09-10 8992 PIZZA FLAME               IBRAHIM MARKET 1770              2   
                                                           1771              2   
5 2021-09-10 9903 CHIL & GRILL 0312.9553655 MOHMAD ZAI     1770              2   
                                                           1771              2   

                                                                 Customer_address  \
3 2021-09-10 9295 BIG BITE FAST FOOD        WARSAK ROAD    1771               375   
4 2021-09-10 8992 PIZZA FLAME               IBRAHIM MARKET 1770               200   
                                                           1771               150   
5 2021-09-10 9903 CHIL & GRILL 0312.9553655 MOHMAD ZAI     1770               200   
                                                           1771               150   

                                                                 I_code  \
3 2021-09-10 9295 BIG BITE FAST FOOD        WARSAK ROAD    1771    9250   
4 2021-09-10 8992 PIZZA FLAME               IBRAHIM MARKET 1770    2700   
                                                           1771    3700   
5 2021-09-10 9903 CHIL & GRILL 0312.9553655 MOHMAD ZAI     1770    2700   
                                                           1771    3700   

                                                                 Product_name  \
3 2021-09-10 9295 BIG BITE FAST FOOD        WARSAK ROAD    1771            10   
4 2021-09-10 8992 PIZZA FLAME               IBRAHIM MARKET 1770            10   
                                                           1771            10   
5 2021-09-10 9903 CHIL & GRILL 0312.9553655 MOHMAD ZAI     1770            10   
                                                           1771            10   

                                                                 CTN_price  \
3 2021-09-10 9295 BIG BITE FAST FOOD        WARSAK ROAD    1771     1925.0   
4 2021-09-10 8992 PIZZA FLAME               IBRAHIM MARKET 1770     1450.0   
                                                           1771     1925.0   
5 2021-09-10 9903 CHIL & GRILL 0312.9553655 MOHMAD ZAI     1770     1450.0   
                                                           1771     1925.0   

                                                                   CTN  PCS  \
3 2021-09-10 9295 BIG BITE FAST FOOD        WARSAK ROAD    1771   75.0    9   
4 2021-09-10 8992 PIZZA FLAME               IBRAHIM MARKET 1770  100.0    9   
                                                           1771   75.0    9   
5 2021-09-10 9903 CHIL & GRILL 0312.9553655 MOHMAD ZAI     1770  100.0    9   
                                                           1771   75.0    9   

                                                                Total_discount  \
3 2021-09-10 9295 BIG BITE FAST FOOD        WARSA

In [4]:
df = df.drop("No", axis=1)

In [5]:
df = df.drop(df['Net_value'].nsmallest(11).index)

min_rows = df.nsmallest(5, ['Net_value'])

# Print the resulting dataframe
print(min_rows)

                                                      Date  C_id  \
11 2021-09-13 9373 BANGASH G/S         DEFENCE 1760  22425     0   
                                               1763  23600     0   
12 2021-09-13 9927 RIYAN GENERAL STORE DEFENCE 1758  20150     0   
                                               1760  22425     0   
                                               1763  23600     0   

                                                     Customer_name  \
11 2021-09-13 9373 BANGASH G/S         DEFENCE 1760              1   
                                               1763              3   
12 2021-09-13 9927 RIYAN GENERAL STORE DEFENCE 1758              2   
                                               1760              2   
                                               1763              2   

                                                     Customer_address  I_code  \
11 2021-09-13 9373 BANGASH G/S         DEFENCE 1760                 0     345   
        

In [6]:
df = df.drop(df['Total_discount'].nsmallest(2).index)

min_rows = df.nsmallest(5, ['Total_discount'])
#df = df.drop(min_rows.index)

# Print the resulting dataframe
print(min_rows)

TypeError: Cannot use method 'nsmallest' with dtype object

In [ ]:
print(df.nunique())

Date                  372
C/code                221
Customer_name         214
Customer_address      131
Item_code              14
Product_name           16
Orignal_price          43
CTN                    10
PCS                    35
Total_discount        332
Discount_price        724
PCS_in_CTN              8
Price_per_PCS          43
Discount_per_piece    252
Month                  12
Price Range             3
Has_Discount            2
dtype: int64


In [ ]:
print('Number of rows:', df.shape[0])
print('Number of columns:', df.shape[1])

Number of rows: 7261
Number of columns: 17
